In [15]:
import numpy as np
import scipy
import scipy.special as sc
import scipy.stats as stats
from scipy._lib.doccer import extend_notes_in_docstring
from scipy.optimize import root_scalar
from scipy.stats import rv_continuous
from scipy.stats._censored_data import CensoredData
from scipy.stats._continuous_distns import _check_fit_input_parameters
from scipy.stats._distn_infrastructure import _ShapeInfo

In [16]:
class weibull_gen(rv_continuous):
    r"""
    DESCRIPTION NEEDS TO BE UPDATED
    Weibull continuous random variable.

    The Weibull Minimum Extreme Value distribution, from extreme value theory
    (Fisher-Gnedenko theorem), is also often simply called the Weibull
    distribution. It arises as the limiting distribution of the rescaled
    minimum of iid random variables.

    %(before_notes)s

    See Also
    --------
    weibull_max, numpy.random.Generator.weibull, exponweib

    Notes
    -----
    The probability density function for `weibull_min` is:

    .. math::

        f(x,k,\lambda) = \frac{k}{\lambda} \left( \frac{x}{\lambda} \right)^{k-1} \exp(-(\frac{x}{\lambda} )^c)

    for :math:`x > 0`, :math:`c > 0`.

    `weibull_min` takes ``c`` as a shape parameter for :math:`c`.
    (named :math:`k` in Wikipedia article and :math:`a` in
    ``numpy.random.weibull``).  Special shape values are :math:`c=1` and
    :math:`c=2` where Weibull distribution reduces to the `expon` and
    `rayleigh` distributions respectively.

    Suppose ``X`` is an exponentially distributed random variable with
    scale ``s``. Then ``Y = X**k`` is `weibull_min` distributed with shape
    ``c = 1/k`` and scale ``s**k``.

    %(after_notes)s

    References
    ----------
    https://en.wikipedia.org/wiki/Weibull_distribution

    https://en.wikipedia.org/wiki/Fisher-Tippett-Gnedenko_theorem

    %(example)s

    """

    def _shape_info(self):
        # tbc: what does integrality & inclusive mean?
        return [
            _ShapeInfo(
                name="k",
                integrality=False,
                domain=(0, np.inf),
                inclusive=(False, False),
            ),
            _ShapeInfo(
                name="lambda_",
                integrality=False,
                domain=(0, np.inf),
                inclusive=(False, False),
            ),
        ]

    def _pdf(self, x, k, lambda_):
        return k / lambda_ * pow(x / lambda_, k - 1) * np.exp(-pow(x / lambda_, k))

    def _logpdf(self, x, k, lambda_):
        return (
            -np.log(lambda_)
            + np.log(k)
            + sc.xlogy(k - 1, x / lambda_)
            - pow(x / lambda_, k)
        )

    def _cdf(self, x, k, lambda_):
        return -sc.expm1(-pow(x / lambda_, k))

    def _ppf(self, q, k, lambda_):
        # point percentile function
        return pow(-sc.log1p(-q), 1.0 / k) * lambda_

    def _sf(self, x, k, lambda_):
        # Survival Function
        # keep the definition to verify logsf, since the numerical integration for logsf
        # fails for k>0 and via this definition, _logsf is still part of the testing
        return np.exp(self._logsf(x, k, lambda_))

    def _logsf(self, x, k, lambda_):
        # log survival function
        return -pow(x / lambda_, k)

    def _isf(self, q, k, lambda_):
        # inverse survival function
        # return -pow(q *lambda_,1/k)
        return pow(-np.log(q), 1 / k) * lambda_

    def _munp(self, n, k, lambda_):
        # Moments of the distribution
        # taken from https://en.wikipedia.org/wiki/Weibull_distribution#Moments -> raw moment
        return pow(lambda_, n) * sc.gamma(1.0 + n * 1.0 / k)

    def _entropy(self, k, lambda_):
        # taken from https://en.wikipedia.org/wiki/Weibull_distribution#Shannon_entropy
        gamma = 0.57721566490153286060  # Euler-Mascheroni constant taken from wiki
        return gamma * (1 - 1 / k) + np.log(lambda_ / k) + 1

    @extend_notes_in_docstring(
        rv_continuous,
        notes="""\
        If ``method='mm'``, parameters fixed by the user are respected, and the
        remaining parameters are used to match distribution and sample moments
        where possible. For example, if the user fixes the location with
        ``floc``, the parameters will only match the distribution skewness and
        variance to the sample skewness and variance; no attempt will be made
        to match the means or minimize a norm of the errors.
        \n\n""",
    )
    def fit(self, data, *args, **kwds):
        if isinstance(data, CensoredData):
            if data.num_censored() == 0:
                data = data._uncensor()
            else:
                return super().fit(data, *args, **kwds)

        if kwds.pop("superfit", False):
            return super().fit(data, *args, **kwds)

        # this extracts fixed shape, location, and scale however they
        # are specified, and also leaves them in `kwds`
        data, fc, floc, fscale = _check_fit_input_parameters(self, data, args, kwds)
        method = kwds.get("method", "mle").lower()

        # See https://en.wikipedia.org/wiki/Weibull_distribution#Moments for
        # moment formulas.
        def skew(c):
            # with k = c
            # this is the original implementation from the weibull_min
            # it does not depend on lambda. Is this  an issue?
            gamma1 = sc.gamma(1 + 1 / c)
            gamma2 = sc.gamma(1 + 2 / c)
            gamma3 = sc.gamma(1 + 3 / c)
            num = 2 * gamma1**3 - 3 * gamma1 * gamma2 + gamma3
            den = (gamma2 - gamma1**2) ** (3 / 2)
            return num / den

        # For c in [1e2, 3e4], population skewness appears to approach
        # asymptote near -1.139, but past c > 3e4, skewness begins to vary
        # wildly, and MoM won't provide a good guess. Get out early.
        s = stats.skew(data)
        max_c = 1e4
        s_min = skew(max_c)
        if s < s_min and method != "mm" and fc is None and not args:
            return super().fit(data, *args, **kwds)

        # If method is method of moments, we don't need the user's guesses.
        # Otherwise, extract the guesses from args and kwds.
        if method == "mm":
            c, loc, scale = None, None, None
        else:
            c = args[0] if len(args) else None
            loc = kwds.pop("loc", None)
            scale = kwds.pop("scale", None)

        if fc is None and c is None:  # not fixed and no guess: use MoM
            # Solve for c that matches sample distribution skewness to sample
            # skewness.
            # we start having numerical issues with `weibull_min` with
            # parameters outside this range - and not just in this method.
            # We could probably improve the situation by doing everything
            # in the log space, but that is for another time.
            c = root_scalar(
                lambda c: skew(c) - s, bracket=[0.02, max_c], method="bisect"
            ).root
        elif fc is not None:  # fixed: use it
            c = fc

        if fscale is None and scale is None:
            v = np.var(data)
            scale = np.sqrt(v / (sc.gamma(1 + 2 / c) - sc.gamma(1 + 1 / c) ** 2))
        elif fscale is not None:
            scale = fscale

        if floc is None and loc is None:
            m = np.mean(data)
            loc = m - scale * sc.gamma(1 + 1 / c)
        elif floc is not None:
            loc = floc

        if method == "mm":
            return c, loc, scale
        else:
            # At this point, parameter "guesses" may equal the fixed parameters
            # in kwds. No harm in passing them as guesses, too.
            return super().fit(data, c, loc=loc, scale=scale, **kwds)


# a is the left end-point (no negative numbers)
weibull = weibull_gen(a=0.0, name="weibull")


In [17]:
class _weibull_tester_gen(rv_continuous):
    """class to run the tests for weibull
    Idea: copy pdf and signature to benchmark others (cdf, sf, ...) via the numerical integration
    of rv_continuous"""

    def __init__(self, *args, **kwargs):
        self.elem = weibull_gen(*args, **kwargs)
        super().__init__(*args, **kwargs)

    def _shape_info(self):
        # tbc: what does integrality & inclusive mean?
        return self.elem._shape_info()

    def _pdf(self, x, k, lambda_):
        return self.elem._pdf(x, k, lambda_)


_weibull_tester = _weibull_tester_gen(a=0.0, name="weibull_tester")

In [18]:
w = weibull(k=2, lambda_=2)
wt = _weibull_tester(k=2, lambda_=2)

In [19]:
k = 2
la = 2
x = 1

w = weibull(k=k, lambda_=la)
wt = _weibull_tester(k=k, lambda_=la)

fun = "isf"

w_result = eval(f"w.{fun}({x})")
wt_result = eval(f"wt.{fun}({x})")
print(f"w={w_result}, wt={wt_result}")

w=0.0, wt=0.0


# Automatic Test

In [27]:
# test this grid. For we want that x or 1/x will represent both odd and even numbers
# for k and lambda
k_list = [1 / 3, 1 / 2, 1, 2, 2.5, 3]
lambda_list = [1 / 3, 1 / 2, 1, 2, 2.5, 3]
function_list = ["cdf", "sf", "logcdf", "isf", "sf"]

x_list = [0.1, 1 / 3, 1 / 2, 0.5, 1, 2, 10, 20]
import itertools

asserter = lambda x, y: np.abs(x - y)
fails = []

eps_max = 1e-3

for fun, k, la, x in itertools.product(function_list, k_list, lambda_list, x_list):
    if fun == "isf" and x > 1:
        continue  # isf is only defined for x<=1

    w = weibull(k=k, lambda_=la)
    wt = _weibull_tester(k=k, lambda_=la)
    w_result = eval(f"w.{fun}({x})")
    wt_result = eval(f"wt.{fun}({x})")
    eps = asserter(w_result, wt_result)

    assert (
        eps < eps_max
    ), f"fun={fun} k={k} la={la} x={x} got w={w_result} & wt={wt_result}"

    if eps > eps_max or np.isnan(w_result):
        # print(f"{fun} x={x} -> eps={eps}")
        print(f"fun={fun} k={k} la={la} x={x} got w={w_result} & wt={wt_result}")
        fails.append(fun)
        # fails.append(f"k={k} la={la} fun={fun} x={x} got w={w_result} & wt={wt_result}")

# Note:
# the numerical integration of the test class fails for logsf for k>1
# Therefore it is excluded from the testing. The SF is implemented as exp(logsf(...))
# --> so it is still verified
assert fails == [], fails

In [149]:
"""Takes long
i_moment = [i+1 for i in range(3)]
# check moments
for k, la,  i in itertools.product(k_list, lambda_list, i_moment):
    w = weibull(k=k, lambda_=la)
    wt = _weibull_tester(k=k, lambda_=la)

    w_moment = w.moment(i)
    wt_moment = wt.moment(i)
    if asserter(w_moment, wt_moment) < eps_max > eps_max:
        print(f"Moment for k={k} la={la} i={i} got w={w_moment} & wt={wt_moment}")

    #assert asserter(w_moment, wt_moment) < eps_max, f"Moment for k={k} la={la} i={i} got w={w_moment} & wt={wt_moment}"
"""

'Takes long\ni_moment = [i+1 for i in range(3)]\n# check moments\nfor k, la,  i in itertools.product(k_list, lambda_list, i_moment):\n    w = weibull(k=k, lambda_=la)\n    wt = _weibull_tester(k=k, lambda_=la)\n\n    w_moment = w.moment(i)\n    wt_moment = wt.moment(i)\n    if asserter(w_moment, wt_moment) < eps_max > eps_max:\n        print(f"Moment for k={k} la={la} i={i} got w={w_moment} & wt={wt_moment}")\n\n    #assert asserter(w_moment, wt_moment) < eps_max, f"Moment for k={k} la={la} i={i} got w={w_moment} & wt={wt_moment}"\n'

In [150]:
# check entropy
for k, la in itertools.product(k_list, lambda_list):
    w = weibull(k=k, lambda_=la)
    wt = _weibull_tester(k=k, lambda_=la)

    w_entropy = w.entropy()
    wt_entropy = wt.entropy()

    assert asserter(w_entropy, wt_entropy) < eps_max

c:\Users\fuerf\AppData\Local\pypoetry\Cache\virtualenvs\scipy-test-MZ1tEUu9-py3.12\Lib\site-packages\scipy\stats\_distn_infrastructure.py:2853: IntegrationWarning: The algorithm does not converge.  Roundoff error is detected
  in the extrapolation table.  It is assumed that the requested tolerance
  cannot be achieved, and that the returned result (if full_output = 1) is 
  the best which can be obtained.
  h = integrate.quad(integ, _a, _b)[0]
c:\Users\fuerf\AppData\Local\pypoetry\Cache\virtualenvs\scipy-test-MZ1tEUu9-py3.12\Lib\site-packages\scipy\stats\_distn_infrastructure.py:2853: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perh

In [136]:
help(w.entropy)

Help on method entropy in module scipy.stats._distn_infrastructure:

entropy() method of scipy.stats._distn_infrastructure.rv_continuous_frozen instance



In [139]:
help(scipy.stats._distn_infrastructure.rv_continuous_frozen)

Help on class rv_continuous_frozen in module scipy.stats._distn_infrastructure:

class rv_continuous_frozen(rv_frozen)
 |  rv_continuous_frozen(dist, *args, **kwds)
 |
 |  Method resolution order:
 |      rv_continuous_frozen
 |      rv_frozen
 |      builtins.object
 |
 |  Methods defined here:
 |
 |  logpdf(self, x)
 |
 |  pdf(self, x)
 |
 |  ----------------------------------------------------------------------
 |  Methods inherited from rv_frozen:
 |
 |  __init__(self, dist, *args, **kwds)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |
 |  cdf(self, x)
 |
 |  entropy(self)
 |
 |  expect(self, func=None, lb=None, ub=None, conditional=False, **kwds)
 |
 |  interval(self, confidence=None)
 |
 |  isf(self, q)
 |
 |  logcdf(self, x)
 |
 |  logsf(self, x)
 |
 |  mean(self)
 |
 |  median(self)
 |
 |  moment(self, order=None)
 |
 |  ppf(self, q)
 |
 |  rvs(self, size=None, random_state=None)
 |
 |  sf(self, x)
 |
 |  stats(self, moments='mv')
 |
 |  std(self)
 |


In [152]:
w.mean()

np.float64(2.678938534707747)